In [1]:
import collections
import os
import pprint
import nltk
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
def extract_features(corpus):
    '''Extract TF-IDF features from corpus'''
    sa_stop_words = nltk.corpus.stopwords.words('english')
    # words that might invert a sentence's meaning so don't remove
    white_list = [
        'what', 'but', 'if', 'because', 'as', 'until', 'against',
        'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again',
        'further', 'then', 'once', 'here', 'there', 'why', 'how', 'all', 'any',
        'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own',
        'same', 'so', 'than', 'too', 'can', 'will', 'just', 'don', 'should']
    stop_words = [sw for sw in sa_stop_words if sw not in white_list]